# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

## Import Packages

In [ ]:
##importing some useful packages
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# import numpy as np
# import cv2
# import os
# %matplotlib inline

## Read in an Image

In [ ]:
##reading in an image
#image = mpimg.imread('test_images/solidWhiteRight.jpg')

##printing out some stats and plotting
#print('This image is:', type(image), 'with dimensions:', image.shape)
#plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
# built-in
import os
import math

# third-party
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

past_neg_lines = np.zeros(shape=(15, 4), dtype=int)
past_pos_lines = np.zeros(shape=(15, 4), dtype=int)
%matplotlib inline

# constants 
apex_ratio = 78/128
center_ratio = 17/32
marg_tr_ratio = 1/128
marg_tl_ratio = 2/32
marg_br_ratio = 13/32
marg_bl_ratio = 16/32

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    #return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image
    
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    # observation:
    # the output must be only two major lines
    # one should have a negative slope \ and
    # one should have a positive slope /
    # combined to be our two lanes /  \
    # the problem is that there will be some that are not solid bars but they should
    # share a similar b (intersection with the height of the image) - we'll have to set
    # a threshold such that we throw out trash lines that could skew our final average
    # differentiate between the two when we get our
    # averages and put precedence on the lines closer to the longest
    # line in that category
    # plan:
    # iterate over lines and group them by slope 
    # iterate over groups and determine prevailing average
    # iterate over groups again and throw out values that 
    # pass a specific threshold
    global past_neg_lines
    global past_pos_lines
    img_height, img_width, _ = img.shape
    midway = img_width * center_ratio
    negatives = []
    positives = []
    
    
    # step 1: iterate over lines and group them by slope
    for cur_line in lines:
        cur_slope = slope(cur_line)
        cx1,cy1,cx2,cy2 = cur_line[0]
        
        if (cur_slope > 0) and all(x > midway for x in (cx1, cx2)): 
            # step 1a: group with postives
            positives.append((cur_line, cur_slope, line_length(cur_line)))
        elif (cur_slope < 0) and all(x < midway for x in (cx1, cx2)): 
            # step 1b: group with negatives
            negatives.append((cur_line, cur_slope, line_length(cur_line)))
        else: 
            # step 2: throw out outliers
            continue
    
    # step 2: average the result lines
    positive_line = average(img, *positives)
    negative_line = average(img, *negatives)
    
    # step 3: average over past frames
    # step 3a: store current frame line
    past_neg_lines = shift_line(
        line=negative_line,
        past_lines=past_neg_lines
    )
    past_pos_lines = shift_line(
        line=positive_line,
        past_lines=past_pos_lines
    )
    # step3b: give average line
    pos_line = best_fit(
        image=img, 
        present_lines=past_pos_lines
    )
    neg_line = best_fit(
        image=img, 
        present_lines=past_neg_lines
    )
        
    # step 3: draw lines
    for line in (l for l in (pos_line, neg_line) if l is not None):
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(
        img=line_img, 
        lines=lines,
        thickness=8
    )
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def mask_vertices(image):
    """
    given an image gives the window of interest as a quadrilateral shape
    :param image: 
    :return: numpy array[[(x1, y1),(x2,y2),(x3,y3),(x4,y4)]]
    """
    imshape = image.shape
    imheight = imshape[0]
    imwidth = imshape[1]
    apex=(imheight)*apex_ratio
    center=(imwidth)*center_ratio
    top_right_margin = center + (imwidth * marg_tr_ratio)
    top_left_margin = center - (imwidth * marg_tl_ratio)
    bottom_right_margin = center + (imwidth * marg_br_ratio)
    bottom_left_margin = center - (imwidth * marg_bl_ratio)
    return np.array(
        [
            [
                (bottom_left_margin, imheight), # BOTTOM LEFT
                (top_left_margin, apex),  # TOP LEFT
                (top_right_margin, apex),  # TOP RIGHT
                (bottom_right_margin, imheight) # BOTTOM RIGHT
            ]
        ],
        dtype=np.int32
    )

def best_fit(image, present_lines):
    """
    purpose: given lines in past frames, attempt to give a
             line that best represents their aggregate
    plan:    many frames contain the same information and each 
             frame may do a better job at recognizing a
             particular line than another.  all lines should
             represent a set of line segment point pairs
             using the point pairs find a line that best
             fits among the "trend" of points
    :param image:
    :param present_lines: numpy.array([[x1a,y1a,x2a,y2a], [x1b,y1b,x2b,y2b], ...])
    """
    mask = np.all(present_lines==0, axis=1)
    # important to remove/mask lines that have all 
    # lines that are [0,0,0,0] as those are just fillers
    # that would throw off our collective best fit line
    # (especially in the beginning frames)
    non_zero_lines = present_lines[~mask]
    # extract x coordinates (x1, x2) from the 1st and 3rd element
    x = non_zero_lines[:, [0, 2]].flatten('F')
    # extract y coordinates (y1, y2) from the 2nd and 4th element
    y = non_zero_lines[:, [1, 3]].flatten('F')
    
    A = np.vstack([x, np.ones(len(x))]).T
    slope, intercept = np.linalg.lstsq(A, y)[0]
    
    return ymxb(
        image=image,
        slope=slope,
        intercept=intercept
    )

def shift_line(line, past_lines):
    """
    an implementation of a circular FIFO memory structure
    to store past line conclusions
    """
    #x1, y1, x2, y2 = line[0]
    if line is None:
        return past_lines
    else:
        present_lines = np.empty_like(past_lines)
        present_lines[:1] = np.array(line[0])
        present_lines[1:] = past_lines[:-1]
        return present_lines
    
def slope(line):
    """
    determine slope from line
    :param line: numpy.array([[x1,y1,x2,y2]])
    :return: float (slope of the line based on rise/run equation)
    """
    x1, y1, x2, y2 = line[0]
    return (y2 - y1)/(x2 - x1)

def intercept(line, slope):
    """
    determines the `b` in `y = mx + b`
    therefore
    y - mx = b
    the y intercept is where y = 0
    0 - mx = b
    -mx = b
    x1, y1 = l, m
    x2, y2 = p, q
    (y2 - y1)/ (x2 - x1) = slope
    """
    x,y,_,_ = line[0]
    return (y - (slope * x))

def ymxb(image, slope, intercept):
    """
    purpose: give largest line that fits inside viewing area
    plan:    given an intercept and slope we can easily
             determine a line that fits in the image dimensions provided
    based on 
    """
    img_height, _, _ = image.shape
    horizon = img_height * apex_ratio
    y1 = img_height
    y2 = horizon
    x1 = (y1 - intercept)/slope
    x2 = (y2 - intercept)/slope
    
    try:
        avg_line = np.array(
            [
                [
                    x1,y1,x2,y2
                ]
            ], 
            dtype=int
        )
    except Exception as e:
        return None
    else:
        return avg_line

def average(image, *samples):
    """
    purpose: give the averaged line for a frame
    plan: as there are many lines that potentially make up
          what I really would consider as one line I give
          a weighted average of both slope (to give average
          orientation of a line) and y intercept (to give 
          average position of a line).  Weight is determined
          by how long of a line is contributing the line
          slope or line intercept as longer lines are more 
          significant in my confidence that a line is more
          than an artifact
    :param image:
    :param samples: different lines sampled on an image 
                    (hopefully from the same target area)
    :return: line | None
    
    """
    # get line lengths and make serve as weight
    
    if not samples:
        return None
    
    weight = sum(
        cur_length
        for (_,_, cur_length) in samples
    )
    
    weighted_average_slope = (
        sum(
            cur_slope*cur_length 
            for (_,cur_slope, cur_length) in samples
        ) 
        / 
        weight
    )
    
    weighted_average_intercept = (
        sum(
            intercept(cur_line, cur_slope)*cur_length
            for (cur_line, cur_slope, cur_length) in samples
        )
        /
        weight
    )
    return ymxb(
        image=image,
        slope=weighted_average_slope,
        intercept=weighted_average_intercept
    )

def line_length(line):
    """
    gives line's length
    :param: np.array([[x1, y1, x2, y2]]) representing the line's endpoints
    :return float:
    """
    x1,y1,x2,y2 = line[0]
    return np.linalg.norm(np.array([x1, y1]) - np.array([x2, y2]))

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images/video
def image_pipeline(image):
    """
    provides an image with lane lines detected
    """
    
    # pipeline
    gray_image = grayscale(image)
    gray_blurred_image = gaussian_blur(
        img=gray_image, 
        kernel_size=3
    )
    edges = canny(
        img=gray_blurred_image,
        low_threshold=50, 
        high_threshold=150
    )
    roi = region_of_interest(
        img=edges, 
        vertices=mask_vertices(edges)
    )
    houghed = hough_lines(
        img=roi, 
        rho=2, 
        theta=np.pi/180, 
        threshold=15, 
        min_line_len=30, 
        max_line_gap=20
    )
    weighted = weighted_img(
        img=houghed,
        initial_img=image
    )
    return weighted

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
# TODO save images ran through the pipeline to the `test_images_output` directory.
original_directory = "test_images"
output_directory = 'test_images_output'
# plt.rcParams["figure.figsize"] = (100, 20)
for file_name in os.listdir(original_directory):
    if not file_name.endswith(".jpg"):
        continue
        
    original_img_path = "{dir}/{file}".format(
        dir=original_directory,
        file=file_name
    )
    image = mpimg.imread(original_img_path)
    converted = image_pipeline(image)
    output_img_path = "{dir}/{file}".format(
        dir=output_directory,
        file=file_name
    )
    mpimg.imsave(output_img_path, converted)
    #cv2.imwrite(output_img_path, converted) output colors are messed up
    plt.imshow(converted)
    plt.show()

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    return image_pipeline(image)

Let's try the one with the solid white lane on the right first ...

In [ ]:
past_neg_pts = np.zeros(shape=(10, 2), dtype=int)
past_pos_pts = np.zeros(shape=(10, 2), dtype=int)
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
past_neg_pts = np.zeros(shape=(10, 2), dtype=int)
past_pos_pts = np.zeros(shape=(10, 2), dtype=int)
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
clip2 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))